<a href="https://colab.research.google.com/github/andersknudby/Teledetection/blob/master/Chapitre_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chapitre 6 - Lire, manipuler et écrire des données matricielles
Les fichiers texte sont relativement faciles à traiter, car ils ont un format simple, et nous savons que nous pouvons traiter toutes les données qu'ils contiennent comme du texte, et qu'elles sont structurées ligne par ligne. La lecture d'un format de fichier plus complexe, comme ceux qui sont généralement utilisés pour stocker des données matricielles, est plus compliquée. Cependant, une grande partie de la complication a été cachée dans des objets et des fonctions déjà créés pour nous dans ce qu'on appelle [GDAL](http://www.gdal.org/) (Geospatial Data Abstraction Library) contient des types d'objets que l'on peut considérer comme des variables complexes, comme un "ensemble de données matricielles" qui possède différentes propriétés telles qu'un "datum" et un "nombre de bandes", etc. GDAL contient également des fonctions permettant de lire, de manipuler et d'écrire de tels objets, de sorte que nous n'avons pas besoin de coder nous-mêmes chaque petit détail de ces opérations complexes.

Un gros inconvénient est que GDAL est curieusement compliqué à importer correctement, surtout sur les machines Windows. Le principal problème est que, pour utiliser GDAL, Python doit savoir où se trouvent tous ses fichiers, et comme GDAL peut être installé de nombreuses façons différentes, et surtout parce qu'un ordinateur peut avoir plusieurs versions de Python installées, cela devient vite très confus. Mais pour l'instant, cela ne va pas nous arrêter, car importer gdal dans Colab est aussi simple que :

In [ ]:
import gdal

Il existe d'autres bibliothèques Python qui utilisent les fonctionnalités de GDAL et ajoutent des fonctions supplémentaires, améliorent la convivialité ou perfectionnent ses fonctionnalités. Nous allons utiliser ici une de ces bibliothèques, appelée rasterio. Dans ce chapitre, nous allons utiliser certaines des fonctions de rasterio, mais nous n'allons pas tout passer en revue. Pour utiliser davantage cette bibliothèque à l'avenir, vous pouvez consulter sa page "readthedocs" [rasterio](https://rasterio.readthedocs.io/en/latest/index.html).

La bibliothèque rasterio n'est pas fournie avec Colab, nous devons donc d'abord l'installer :

In [ ]:
!pip install rasterio

Et quand on aura fini de l'installer, on pourra l'importer :

In [ ]:
import rasterio

Pour ce chapitre, nous utiliserons une section d'une orthophoto aérienne de l'Université Simon Fraser, enregistrée au format GeoTiff, appelée 'sfu.tif'. Si vous voulez avoir une idée de ce à quoi ressemble l'image avant de travailler avec elle en Python (c'est toujours une bonne idée), ouvrez-la dans un des logiciels que vous connaissez déjà et jetez un coup d'œil à ses propriétés, etc. QGIS est idéal pour cela, car vous pouvez l'installer sur votre ordinateur personnel, quel que soit son système d'exploitation. Si vous faites cela, vous remarquerez que la "première bande" de l'image (par exemple, ce que ArcGIS appelle "Bande_1") est la bande "rouge". En d'autres termes, elle contient des informations sur la quantité de rayonnement électromagnétique dans la gamme de longueurs d'onde 600-700nm atteignant la caméra depuis différentes parties du campus. De même, la deuxième bande est le "vert", et la troisième le "bleu". Si nous voulons utiliser cette image de manière intelligente, nous devons savoir ceci.

##Lecture d'un jeu de données matricielles
GDAL structure les données matricielles dans une hiérarchie, avec trois composants principaux.

1\) Le **jeu de données** est l'ensemble du jeu de données, y compris toutes les données et toutes les métadonnées dans un seul objet. Le jeu de données possède certaines propriétés, comme une projection, un datum, une géotransformation (qui contient par exemple la taille du pixel), etc. Elles sont toutes définies pour le jeu de données car elles sont une propriété du jeu de données - elles doivent être les mêmes pour toutes les couches de données matricielles individuelles dans un jeu de données.

2\) Le jeu de données est organisé en **bandes**. Les bandes ont également certaines propriétés, comme la valeur minimale et maximale. De même, ces propriétés sont définies pour chaque bande car elles sont une propriété de la bande, et non du jeu de données, et non du pixel individuel. Cependant, il est important de noter qu'une bande dans GDAL est un "pointeur" vers les données qui sont dans la bande elle-même, de sorte que vous pouvez "ouvrir la bande" pour accéder à ses propriétés (par exemple, sa valeur minimale) sans avoir à lire toutes les données. Cela peut être déroutant pour l'instant, mais ce sera plus clair dans les exemples qui suivent.

3\)	Les données réelles désignées par chaque bande peuvent être lues et stockées en Python sous forme de **arrays** NumPy. Ces arrays ont toutes les mêmes propriétés que les autres arrays NumPy, comme la forme (par exemple, le nombre de colonnes et de lignes).

rasterio hérite également de cette structure, bien qu'il fusionne les **bandes** et les **arrays** dans ses propres objets de données. Un exemple permettra d'illustrer comment tout cela fonctionne. Tout d'abord, nous devons nous assurer que nous avons accès au fichier via Google Drive :

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

myDir = '/content/drive/My Drive/Python files/'

import os
if os.path.exists(myDir + 'sfu.tif'):
  print("Drive mounted and directory found")
else:
  print("No access to the files")

Ensuite, nous allons ouvrir le fichier avec rasterio :

In [ ]:
fileName = myDir + 'sfu.tif'

ds = rasterio.open(fileName)  # ds is a commonly used shorthand for 'dataset'

Nous disposons maintenant d'un ensemble de données, nous pouvons donc en tirer quelques enseignements :

In [ ]:
print("Dataset name is", ds.name)
print("Number of bands in dataset:", ds.count)
print("Number of columns in dataset:", ds.width)
print("Number of rows in dataset:", ds.height)

Nous pouvons également obtenir des informations sur la géoréférence des données :

In [ ]:
print("Dataset coordinate reference system (CRS):", ds.crs)
print("Dataset bounds:", ds.bounds)
print("Dataset geotransform:", ds.transform)

* Le code EPSG que vous voyez ci-dessus fait référence à la zone UTM 10N, basée sur le datum WGS84.
* Les limites nous indiquent la zone couverte par l'image.
* Et la transformation nous indique la taille du pixel (0,10, en mètres), la coordonnée x du côté gauche de l'image (506537.0) et la coordonnée y du haut de l'image (5458686.0).
* Toutes les coordonnées des limites et de la transformation se réfèrent au CRS.

Vous ne l'avez peut-être pas remarqué, mais l'ouverture du jeu de données a été très rapide - beaucoup plus rapide que si vous l'aviez ouvert dans un logiciel SIG. C'est parce que Python n'a pas réellement lu toutes les données - il a juste lu toutes les informations sur les données, comme ce que nous avons imprimé dans les blocs de code ci-dessus. Pour lire les données réelles, nous devons lire les bandes, et à partir des bandes, nous pouvons lire les arrays - les valeurs numériques réelles dont l'image est composée.

**Remarque importante : GDAL commence à compter les bandes à partir de 1 (et non 0, comme c'est le cas par défaut en Python) :

In [ ]:
band1 = ds.read(1)
band1

Les valeurs de l'array, c'est-à-dire les valeurs des pixels individuels de l'image, peuvent être adressées par leur index de ligne et de colonne :

In [ ]:
band1[453, 1243]

Nous savons maintenant comment ouvrir un ensemble de données d'image, obtenir des informations à son sujet et lire les valeurs des bandes individuelles dans des arrays NumPy. Ce qui signifie que nous pouvons utiliser notre connaissance des arrays NumPy pour travailler avec les données !

Par exemple, si nous voulons décrire la luminosité de chaque pixel indépendamment de sa "couleur", nous pouvons la calculer comme, par exemple, la moyenne de sa valeur dans les trois bandes. Et nous pouvons utiliser les fonctions de array de NumPy pour le faire très rapidement et efficacement :

In [ ]:
import numpy as np
band1 = ds.read(1)
band2 = ds.read(2)
band3 = ds.read(3)
brightness = (band1 + band2 + band3) / 3
brightness

**Avertissement:** Il y a une chose à considérer ici, que GDAL (et rasterio, et NumPy) n'est pas particulièrement bien construit pour nous aider. Il y a en fait une erreur importante - une erreur sémantique - dans le code ci-dessus. La façon la plus simple de l'illustrer est de regarder un seul pixel (et de prêter attention au 'RuntimeWarning' qu'il affiche lorsque nous exécutons le code ci-dessous :

In [ ]:
band1Value = band1[100,500]
band2Value = band2[100,500]
band3Value = band3[100,500]
print("Values in the three bands are:", band1Value, band2Value, band3Value)

averageValue = (band1Value + band2Value + band3Value) / 3
print("The average value is:", averageValue)

Que se passe-t-il ici - les trois valeurs sont respectivement 131, 129 et 111, et lorsque nous calculons la moyenne, elle est de... 38.3 ?!?

Le problème est que les données de l'image sont stockées sous forme d'entiers non signés de 8 bits, comme le montre l'exemple suivant :

In [ ]:
band1Value.dtype

N'hésitez pas à faire des recherches pour en savoir plus sur les types de données, mais la version courte pertinente ici est que les entiers non signés de 8 bits ne peuvent contenir que des valeurs comprises entre 0 et 255. Lorsque nous calculons (band1Value + band2Value + band3Value), nous obtenons 131 + 129 + 111 = 371. C'est plus que ce que notre type de données peut gérer. Ce que NumPy fait alors, c'est que lorsqu'il compte et atteint 255, il se replie et recommence à compter à partir de 0 (au lieu de 256). Ainsi, étant donné qu'il est lié à un entier non signé de 8 bits, au lieu d'ajouter toutes les données jusqu'à 371, il arrive à 371 - 256 = 115. Et ***ensuite*** il divise par 3, pour arriver à 38,3.

Pour éviter toutes ces absurdités, la solution la plus simple consiste à convertir les données en un type de données plus approprié lorsque vous les lisez pour la première fois dans le fichier image. Dans notre exemple, nous pouvons le faire comme suit, avec la fonction 'astype' de NumPy :

In [ ]:
import numpy as np
band1 = ds.read(1).astype('uint16')
band2 = ds.read(2).astype('uint16')
band3 = ds.read(3).astype('uint16')
brightness = (band1 + band2 + band3) / 3

Et de prouver qu'il produit le résultat souhaité :

In [ ]:
band1Value = band1[100,500]
band2Value = band2[100,500]
band3Value = band3[100,500]
print("Values in the three bands are:", band1Value, band2Value, band3Value)

averageValue = (band1Value + band2Value + band3Value) / 3
print("The average value is:", averageValue)

##Écriture d'un jeu de données matricielles
Maintenant que nous avons un produit de notre analyse d'image, nous voulons généralement écrire le résultat dans un fichier comme un nouveau jeu de données matricielles. L'écriture des arrays NumPy dans des fichiers matriciels comprend une série d'étapes, comme indiqué ci-dessous. C'est plus compliqué que d'écrire un fichier texte, car les ensembles de données matricielles contiennent non seulement les données d'image, mais aussi les informations associées que nous avons lues précédemment, comme le système de référence des coordonnées, les limites, etc. Les différents composants des données, tels que listés dans les newDs ci-dessous, sont :
* myDir + 'brightness.tif' indique le nom du fichier à créer et à écrire.
* w' indique que nous voulons que ce fichier soit ouvert à l'écriture.
* driver='GTiff' indique que nous voulons que le format du fichier soit GeoTiff.
* height et width sont le nombre de lignes et de colonnes de la nouvelle image.
* count est le nombre de bandes dans la nouvelle image. Alors que nous en avions trois dans l'original, nous n'en avons qu'une ici (pour écrire la luminosité dans)
* dtype='float64' indique que nous voulons écrire des valeurs décimales dans les pixels individuels.
* crs et transform sont les mêmes que l'image originale

In [ ]:
newDs = rasterio.open(myDir + 'brightness.tif', 'w', driver='GTiff',
                            height=ds.height, width=ds.width, count=1,
                            dtype='float64', crs=ds.crs, transform=ds.transform)
newDs.write(brightness, 1)
newDs.close()

Comme dans le dernier chapitre, pour écrire réellement ce fichier sur votre Google Drive, vous devez le vider et le démonter :

In [ ]:
drive.flush_and_unmount()

Allez sur votre Google Drive, téléchargez le fichier appelé "brightness.tif", et ouvrez-le dans QGIS à côté du fichier image original. Le résultat est-il logique, c'est-à-dire que les pixels brillants de l'image originale ont-ils des valeurs plus élevées dans l'image de luminosité ?

##Exercice
Pour cartographier la végétation avec une imagerie à trois bandes (rouge, verte et bleue), vous pouvez vous appuyer sur le fait que la plupart des choses autres que la végétation, dans le monde naturel, ne sont pas vertes. Vous pouvez donc calculer un indice de "verdure" pour représenter la végétation. Un indice souvent utilisé est la coordonnée chromatique verte (CCV), qui se calcule simplement comme suit : Vert / (Rouge + Vert + Bleu). Modifiez le code de ce chapitre pour calculer le GCC, et écrivez-le dans un nouveau fichier appelé 'gcc.tif'. Comparez-le à l'image originale pour vérifier que les valeurs élevées du CCG correspondent bien à des zones de végétation, et vice versa.